<a href="https://colab.research.google.com/github/vigneshwaran-dev/CV-research-timeline/blob/main/GoogLeNet/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Lambda, Input, GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.nn import local_response_normalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler

Defining the Model as per the Original Paper

In [3]:
def inception_block(model, filter_map):
  branch_0 = Conv2D(filters=filter_map[0], kernel_size=(1, 1), padding='same')(model)
  branch_0 = Activation('relu')(branch_0)
  
  branch_1 = Conv2D(filters=filter_map[1][0], kernel_size=(1, 1), padding='same')(model)
  branch_1 = Activation('relu')(branch_1)
  branch_1 = Conv2D(filters=filter_map[1][1], kernel_size=(1, 1), padding='same')(branch_1)
  branch_1 = Activation('relu')(branch_1)
  
  branch_2 = Conv2D(filters=filter_map[2][0], kernel_size=(1, 1), padding='same')(model)
  branch_2 = Activation('relu')(branch_2)
  branch_2 = Conv2D(filters=filter_map[2][1], kernel_size=(1, 1), padding='same')(branch_2)
  branch_2 = Activation('relu')(branch_2)
  
  branch_3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(model)
  branch_3 = Conv2D(filters=filter_map[3], kernel_size=(1, 1), padding='same')(branch_3)
  branch_3 = Activation('relu')(branch_3)
  
  return tf.concat([branch_0, branch_1, branch_2, branch_3], axis=3)

# Initial Convolutional Block
_input = Input(shape=(224, 224, 3))
model = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same')(_input)
model = Activation('relu')(model)
model = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(model)
model = Lambda(local_response_normalization)(model)
model = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same')(model)
model = Activation('relu')(model)
model = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), padding='same')(model)
model = Activation('relu')(model)
model = Lambda(local_response_normalization)(model)
model = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(model)

# 1st Sequence of Inceptions
model = inception_block(model, [64, [96, 128], [16, 32], 32])
model = inception_block(model, [128, [128, 192], [32, 96], 64])
model = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(model)
model = inception_block(model, [192, [96, 208], [16, 48], 64])

# 1st Output Branch
output_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3))(model)
output_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='same')(output_1)
output_1 = Activation('relu')(output_1)
output_1 = Flatten()(output_1)
output_1 = Dense(1024)(output_1)
output_1 = Activation('relu')(output_1)
output_1 = Dropout(0.7)(output_1)
output_1 = Dense(1000)(output_1)
output_1 = Activation('softmax')(output_1)

# 2nd Sequence of Inceptions
model = inception_block(model, [160, [112, 224], [24, 64], 64])
model = inception_block(model, [128, [128, 256], [24, 64], 64])
model = inception_block(model, [112, [144, 288], [32, 64], 64])

# 2nd Output Branch
output_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3))(model)
output_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='same')(output_2)
output_2 = Activation('relu')(output_2)
output_2 = Flatten()(output_2)
output_2 = Dense(1024)(output_2)
output_2 = Activation('relu')(output_2)
output_2 = Dropout(0.7)(output_2)
output_2 = Dense(1000)(output_2)
output_2 = Activation('softmax')(output_2)

# 3rd Sequence of Inceptions
model = inception_block(model, [256, [160, 320], [32, 128], 128])
model = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(model)
model = inception_block(model, [256, [160, 320], [32, 128], 128])
model = inception_block(model, [384, [192, 384], [48, 128], 128])

# Final Output
model = GlobalAveragePooling2D()(model)
model = Dropout(0.4)(model)
model = Dense(1000)(model)
model = Activation('softmax')(model)

model = Model(inputs=_input, outputs=[model, output_1, output_2])

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 55, 55, 64)   0           activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(loss=[categorical_crossentropy, categorical_crossentropy, categorical_crossentropy],
                 optimizer=SGD(learning_rate=0.01, momentum=0.9),
                 metrics=['accuracy'])

Considering the data to be present in TRAIN_DATA_LOCATION and VALIDATION_DATA_LOCATION directories and running them through data generators to perform live data augumentation during the training process

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'TRAIN_DATA_LOCATION'
valid_dir = 'VALIDATION_DATA_LOCATION'

BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=1,
                                                    shuffle=True,
                                                    class_mode='categorical')

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=BATCH_SIZE,
                                                    seed=7,
                                                    shuffle=True,
                                                    class_mode='categorical')

train_num = train_generator.samples

Training the Model

In [ ]:
import datetime

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callback_list = [tensorboard_callback]

def reduce_lr(epoch):
  lr = 0.01
  reduction = epoch % 8
  
  for _ in range(1, reduction):
    lr -= (lr / 100) * 80
  
  return lr

callback_list.append(LearningRateScheduler(reduce_lr))

model.fit(train_generator,
          epochs=1,
          steps_per_epoch=train_num // BATCH_SIZE,
          validation_data=valid_generator,
          validation_steps=valid_num // BATCH_SIZE,
          callbacks=callback_list,
          verbose=1)

model.save('googlenet.h5')

Visualizing the performance using Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Prediction

In [ ]:
x_valid, label_batch  = next(iter(valid_generator))
prediction_values = model.predict_classes(x_valid)
print(prediction_values)